In [4]:
#import libraries 
import pandas as pd
import numpy as np

# SAN FRANCISCO CRIME DATA 2018

Dataset includes police incident reports in San Francisco. Reports included are those for incidents that occurred starting January 1, 2018.

## AIM OF ANALYSIS

The description of the crimes is varied and the objective of this analysis is to be able to identify if crimes with the same description have a correlation with a particular area in order to act in the future in a preventive manner. 

To make this analysis and to group crimes based on different characteristics we can use different types of cluster analysis which we will analyze later.

We don't know if exist any common group of crime wich are related on particular characteristics and we can't assert which number of group is more efficient to do on future an analysis to prevents them. 

## DATA DESCRIPTION 
We begin the work by describing in detail the available dataset.

Dataset has 533 rows and 22 columns. Each crime is identified by specific ID and Case Number of reports submitted by an agent. The variables 'Description', 'Name of District' and the location (indates with two variables 'Community Area' and 'Location') are fundamentals to do a cluster analysis.
The coordinates can be used to build a map.

In [54]:
crimedata = pd.read_csv('Chicago_Crime_Data.csv')
crimedata.head(2)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,0890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,14.0,58.0,06,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,0820,THEFT,$500 AND UNDER,OTHER,False,False,...,27.0,23.0,06,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"


In [55]:
crimedata.shape #533 rows and 22 columns

(533, 22)

In [56]:
crimedata.columns #name of columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [57]:
crimedata.dtypes

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

In [58]:
print(len(np.unique(crimedata['Primary Type']))) #35 type of different crime
np.unique(crimedata['Primary Type']) #name of crime

35


array(['ARSON', 'ASSAULT', 'BATTERY', 'BURGLARY',
       'CONCEALED CARRY LICENSE VIOLATION', 'CRIM SEXUAL ASSAULT',
       'CRIMINAL DAMAGE', 'CRIMINAL TRESPASS', 'DECEPTIVE PRACTICE',
       'DOMESTIC VIOLENCE', 'GAMBLING', 'HOMICIDE', 'HUMAN TRAFFICKING',
       'INTERFERENCE WITH PUBLIC OFFICER', 'INTIMIDATION', 'KIDNAPPING',
       'LIQUOR LAW VIOLATION', 'MOTOR VEHICLE THEFT', 'NARCOTICS',
       'NON - CRIMINAL', 'NON-CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'OBSCENITY',
       'OFFENSE INVOLVING CHILDREN', 'OTHER NARCOTIC VIOLATION',
       'OTHER OFFENSE', 'PROSTITUTION', 'PUBLIC INDECENCY',
       'PUBLIC PEACE VIOLATION', 'RITUALISM', 'ROBBERY', 'SEX OFFENSE',
       'STALKING', 'THEFT', 'WEAPONS VIOLATION'], dtype=object)

In [59]:
crimedata.index = crimedata.ID      

## Cluster analysis 
### K-Means

In [32]:
from sklearn.cluster import KMeans
import pickle 
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

In [60]:
crimedata.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
ID,,,,,,,,,,,,,,,,,,,,,
3512276,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,0890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,14.0,58.0,06,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
3406613,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,0820,THEFT,$500 AND UNDER,OTHER,False,False,...,27.0,23.0,06,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
8002131,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,0820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,3.0,38.0,06,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
7903289,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,7.0,46.0,06,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
10402076,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,0820,THEFT,$500 AND UNDER,ALLEY,False,False,...,15.0,66.0,06,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [66]:
crimedata = crimedata[['X Coordinate','Y Coordinate','Latitude', 'Longitude']]
crimedata = crimedata.dropna()
crimedata.head()

,X Coordinate,Y Coordinate,Latitude,Longitude
ID,,,,
3512276,1155838.0,1873050.0,41.807441,-87.703956
3406613,1152206.0,1906127.0,41.898280,-87.716406
8002131,1177436.0,1876313.0,41.815933,-87.624642
7903289,1194622.0,1850125.0,41.743665,-87.562463
10402076,1155240.0,1860661.0,41.773455,-87.706480


In [67]:
crimedata.isna().sum()

X Coordinate    0
Y Coordinate    0
Latitude        0
Longitude       0
dtype: int64

In [69]:
cluster = pd.DataFrame()
crimedata_array=np.asarray(crimedata)
km= KMeans(n_clusters=2,n_init=10, max_iter=300, tol=1e-04, random_state=0)
y_km = km.fit_predict(crimedata_array)
df_feat = pd.DataFrame(crimedata_array)
crimedata_array = crimedata_array.assign(label = y_km)
cluster  = pd.concat([cluster , crimedata_array])

AttributeError: 'numpy.ndarray' object has no attribute 'assign'